In [ ]:
# Import libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import wilcoxon

In [ ]:
# Define a general dataframe to hold all data
all_data = pd.DataFrame()

for i in range(10):
    # Create a directory for each run
    data_file_path = f'results/run{i}/data.csv'

    # Read the CSV file into a pandas dataframe
    df = pd.read_csv(data_file_path)

    # Add a new column 'run' with the value of i
    df['run'] = i

    # Append the dataframe to the all_data dataframe
    all_data = pd.concat([all_data, df], ignore_index=True)

print (all_data.head())

In [ ]:
# We need to fill the dataframe. For each asm and each run, we need values with approach = evoavalla, approach = random, and approach = atgt. If they are not present, we need to fill them with NaN
asms = all_data['asm'].unique()
approaches = all_data['approach'].unique()

for asm in asms:
    for run in range(10):
        for approach in approaches:
            # Check if the combination exists
            if not ((all_data['asm'] == asm) & (all_data['run'] == run) & (all_data['approach'] == approach)).any():
                # If it doesn't exist, create a new row with NaN values
                new_row = {
                    'asm': asm,
                    'run': run,
                    'approach': approach,
                    'n_failing_scenarios': float('nan'),
                    'update_rule_coverage': float('nan'),
                    'branch_coverage': float('nan'),
                    'macro_coverage': float('nan'),
                    'n_checks': float('nan'),
                    'n_sets': float('nan'),
                    'n_step': float('nan'),
                    'n_scenarios': 0
                }
                # all_data = all_data.append(new_row, ignore_index=True) # append removed since pandas 2.0
                all_data = pd.concat([all_data, pd.DataFrame([new_row])], ignore_index=True)

In [ ]:
# Rename approach atgt to ATGT
all_data['approach'] = all_data['approach'].replace('atgt', 'ATGT')
# Rename approach evoavalla to EvoAValla
all_data['approach'] = all_data['approach'].replace('evoavalla', 'EvoAvalla')
# Rename approach random to Random
all_data['approach'] = all_data['approach'].replace('random', 'Random')
# Remove all lines where 'asm' is Ferryman
# all_data = all_data[all_data['asm'] != 'Ferryman']
# Print the number of distinct asms
print("Number of distinct asms: ", all_data['asm'].nunique())

In [ ]:
# For each approach, calculate the mean and standard deviation of the 'exec_time' column    
for approach in all_data['approach'].unique():
    # Filter the dataframe for the current approach
    approach_data = all_data[all_data['approach'] == approach]

    # Calculate mean and standard deviation
    mean_exec_time = approach_data['exec_time'].mean()
    std_exec_time = approach_data['exec_time'].std()

    # Print the results
    print(f"Approach: {approach}, Mean Execution Time: {mean_exec_time:.2f}, Std Dev: {std_exec_time:.2f}")

# Generate a boxplot for the execution times of each approach
# Set the style of seaborn
sns.set(style="whitegrid")
# Create a boxplot
plt.figure(figsize=(4, 3))
# EvoAvalla must be blue, Random must be orange, and ATGT must be green
sns.set_palette(['#1f77b4', '#ff7f0e', '#2ca02c'])
sns.boxplot(x='approach', y='exec_time', data=all_data)
# set log scale for y-axis
plt.yscale('log')
plt.xlabel('')
plt.ylabel('Time [s]')
plt.tight_layout()

# Save the image as pdf
plt.savefig('images/boxplot_exec_time.pdf', bbox_inches='tight')

In [ ]:
# For each approach, calculate the mean and standard deviation of the 'branch_coverage' column
for approach in all_data['approach'].unique():
    # Filter the dataframe for the current approach
    approach_data = all_data[all_data['approach'] == approach]

    # Calculate mean and standard deviation
    mean_branch_coverage = approach_data['branch_coverage'].mean()
    std_branch_coverage = approach_data['branch_coverage'].std()

    # Print the results
    print(f"Approach: {approach}, Mean Branch Coverage: {mean_branch_coverage:.2f}, Std Dev: {std_branch_coverage:.2f}")

In [ ]:
# For each approach, calculate the mean and standard deviation of the 'macro_coverage' column
for approach in all_data['approach'].unique():
    # Filter the dataframe for the current approach
    approach_data = all_data[all_data['approach'] == approach]

    # Calculate mean and standard deviation
    mean_macro_coverage = approach_data['macro_coverage'].mean()
    std_macro_coverage = approach_data['macro_coverage'].std()

    # Print the results
    print(f"Approach: {approach}, Mean Macro Coverage: {mean_macro_coverage:.2f}, Std Dev: {std_macro_coverage:.2f}")

In [ ]:
# For each approach, calculate the mean and standard deviation of the 'update_rule_coverage' column
for approach in all_data['approach'].unique():
    # Filter the dataframe for the current approach
    approach_data = all_data[all_data['approach'] == approach]

    # Calculate mean and standard deviation
    mean_update_rule_coverage = approach_data['update_rule_coverage'].mean()
    std_update_rule_coverage = approach_data['update_rule_coverage'].std()

    # Print the results
    print(f"Approach: {approach}, Mean Update Rule Coverage: {mean_update_rule_coverage:.2f}, Std Dev: {std_update_rule_coverage:.2f}")

In [ ]:
# For each approach, calculate the mean and standard deviation of the 'n_scenarios' column
for approach in all_data['approach'].unique():
    # Filter the dataframe for the current approach
    approach_data = all_data[all_data['approach'] == approach]

    # Calculate mean and standard deviation
    mean_n_scenarios = approach_data['n_scenarios'].mean()
    std_n_scenarios = approach_data['n_scenarios'].std()

    # Print the results
    print(f"Approach: {approach}, Mean Number of Scenarios: {mean_n_scenarios:.2f}, Std Dev: {std_n_scenarios:.2f}")

In [ ]:
# For each approach, calculate the mean and standard deviation of the 'n_step' column
for approach in all_data['approach'].unique():
    # Filter the dataframe for the current approach
    approach_data = all_data[all_data['approach'] == approach]

    # Calculate mean and standard deviation
    mean_n_step = approach_data['n_step'].mean()
    std_n_step = approach_data['n_step'].std()

    # Print the results
    print(f"Approach: {approach}, Mean Number of Steps: {mean_n_step:.2f}, Std Dev: {std_n_step:.2f}")

In [ ]:
# For each approach, calculate the number of distinc asm for which at least one run produced n_scenarios = 0 or n_scenarios = NaN
for approach in all_data['approach'].unique():
    # Filter the dataframe for the current approach
    approach_data = all_data[all_data['approach'] == approach]

    # Get the unique asms for which at least one run produced n_scenarios = 0 or n_scenarios = NaN
    unique_asms = approach_data[(approach_data['n_scenarios'] == 0) | (approach_data['n_scenarios'].isna())]['asm'].unique()

    # Print the results
    print(f"Approach: {approach}, Number of unique asms with n_scenarios = 0 or NaN: {len(unique_asms)} - {unique_asms}")

# Create an histogram showing the number of asms for each technique for which at least one run produced n_scenarios = 0 or n_scenarios = NaN
plt.figure(figsize=(4, 4))
sns.histplot(data=all_data[(all_data['n_scenarios'] == 0) | (all_data['n_scenarios'].isna())], x='asm', hue='approach', multiple='stack')
plt.ylabel('Number of failing runs')
plt.xticks(rotation=90)
plt.xlabel('')
plt.tight_layout()
# Set the legend 
plt.legend(title='Approach', loc='upper right', labels=['ATGT', 'Random', 'EvoAvalla'], ncols=2)

# Save the image as pdf
plt.savefig('images/histogram_asm_zero_scenarios.pdf', bbox_inches='tight')

In [ ]:
# Generate a box-plot with the update rule, macro and branch coverage for all three approaches
sns.set(style="whitegrid")
# Create a boxplot
plt.figure(figsize=(4, 3))
# EvoAvalla must be blue, Random must be orange, and ATGT must be green
sns.set_palette(['#1f77b4', '#ff7f0e', '#2ca02c'])
sns.boxplot(x='approach', y='branch_coverage', data=all_data)
# set log scale for y-axis
plt.xlabel('')
plt.ylabel('Branch Coverage')
plt.tight_layout()

# Save the image as pdf
plt.savefig('images/boxplot_branch_coverage.pdf', bbox_inches='tight')

In [ ]:
sns.set(style="whitegrid")
# Create a boxplot
plt.figure(figsize=(4, 3))
# EvoAvalla must be blue, Random must be orange, and ATGT must be green
sns.set_palette(['#1f77b4', '#ff7f0e', '#2ca02c'])
sns.boxplot(x='approach', y='macro_coverage', data=all_data)
# set log scale for y-axis
plt.xlabel('')
plt.ylabel('Macro Coverage')
plt.tight_layout()

# Save the image as pdf
plt.savefig('images/boxplot_macro_coverage.pdf', bbox_inches='tight')

In [ ]:
sns.set(style="whitegrid")
# Create a boxplot
plt.figure(figsize=(4, 3))
# EvoAvalla must be blue, Random must be orange, and ATGT must be green
sns.set_palette(['#1f77b4', '#ff7f0e', '#2ca02c'])
sns.boxplot(x='approach', y='update_rule_coverage', data=all_data)
# set log scale for y-axis
plt.xlabel('')
plt.ylabel('Update Rule Coverage')
plt.tight_layout()

# Save the image as pdf
plt.savefig('images/boxplot_updaterule_coverage.pdf', bbox_inches='tight')